In [1]:
from eth_account import Account
import time
import os
import sys
from pathlib import Path
import time
path = os.path.abspath('')
parent_dir = str(Path(path).parents[1])
print(parent_dir)
sys.path.append(parent_dir)
from pyledger.extras.evmnet.participant_scripts import *
#from pyledger.examples.padl_deployERC import load_issuer_secret_key

/Users/shaltieleloul/PADL
/Users/shaltieleloul/PADL/pyledger/extras/evmnet/ip_config.ini


In [2]:
# Account setup 
example_private_key = '0x8f2a55949038a9610f50fb23b5883af3b4ecb3c3bb792cbcefbd1542c692be63'
LocalAccount = Account.from_key(example_private_key)

In [24]:
LocalAccount.address

'0xFE3B557E8Fb62b89F4916B721be55cEb828dBd73'

## Step 1 is to deploy smart contract

In [3]:
contract_address = deploy_new_contract(example_private_key, v0=[1000,1000], types={'0': 'x', '1': 'y'})
print(contract_address)

0xDF3149B11d7457eA472a56463bB9c0928fC25946


In [4]:
ledger, issuer = get_ledger_bank_padl("Issuer 0")
public_key = issuer.pk

## Step 2: Contract deployed; Now participants request access to contract;

In [5]:
# BANK 1 registers to PADL 
bank1_account_dict = create_account(contract_address)
bank1_address = bank1_account_dict['address']
add_participant(bank1_address)
register_padl(name="Bank", v0=[0,0], types={'0': 'x', '1': 'y'}, account_dict=bank1_account_dict, audit_pk =public_key)
bank_send_deposit(bank1_account_dict, 0)
check_balance("Bank 1")

new account created with address 0xfb0F343D60113C9a41180F628c830AAff3517d6B
address 0xfb0F343D60113C9a41180F628c830AAff3517d6B
Eth balance for address 0xfb0F343D60113C9a41180F628c830AAff3517d6B is 99.99781867
Sending deposit
Deposit sent
Eth balance for address 0xfb0F343D60113C9a41180F628c830AAff3517d6B is 99.997797622
checking contract Eth balance
Contract Eth balance at address 0xDF3149B11d7457eA472a56463bB9c0928fC25946 is 0
------- Bank 1 -------
x 0
y 0
-----------------------


[0, 0]

In [6]:
# BANK 2 registers to PADL 
bank2_account_dict = create_account(contract_address)
bank2_address = bank2_account_dict['address']
add_participant(bank2_address)
register_padl(name="Bank", v0=[0,0], types={'0': 'x', '1': 'y'}, account_dict=bank2_account_dict, audit_pk =public_key)
bank_send_deposit(bank2_account_dict, 0)
check_balance("Bank 2")

new account created with address 0x42571e717FDA48370e2442D0DdB708656a8837D8
address 0x42571e717FDA48370e2442D0DdB708656a8837D8
Eth balance for address 0x42571e717FDA48370e2442D0DdB708656a8837D8 is 99.997818694
Sending deposit
Deposit sent
Eth balance for address 0x42571e717FDA48370e2442D0DdB708656a8837D8 is 99.997797646
checking contract Eth balance
Contract Eth balance at address 0xDF3149B11d7457eA472a56463bB9c0928fC25946 is 0
------- Bank 2 -------
x 0
y 0
-----------------------


[0, 0]

## Issuer sends initial assets as private transaction to both participants with on-chain verification 

In [7]:
send_injective_tx(vals=[[-4, 0, 4],[-4, 4, 0]], file_name="Issuer 0")

Preparing transaction
sending transaction
gas used for injective tx of asset 0, and no. of parties 3: 7169075
sending transaction
gas used for injective tx of asset 1, and no. of parties 3: 6643753


([[{'cm': (16364624204569708780397493526860578434615030615009879481773775749149061501155,
     21310510025941983392802936878453093197580299203264902008363870351814427369351),
    'tk': (1424280754745699114323997764495650203660120972820927328541924988893686718258,
     1852243541732226585460983734204037857276669952279027559929787171170653664664),
    'compcm': (10152848215660418839786867919601081715656232445412746578115309784872324133432,
     13789719562826089597320983512609482653883119593003804016525517471620034599962),
    'comptk': (348099841880675479784325637995099577692492838230279973416264018187994734396,
     18240835457189848333109863672369514856644321303131623651304597313047773333),
    'ppositive': ((17259894809648788803890536293571184510417765790626118129733711094777942123837,
      10839812639543367035698444414553227421035641969430621500502350051880570031942),
     (8364643014663884037090109297457531679585887132238082938408050619732953269648,
      3549168879442978333196303

In [14]:
#tx = send_coins(vals=[ [-2, 2, 0],[1, -1, 0] ], file_name="Issuer 0", audit_pk = public_key ) # this is the broadcast  (tx + approve tx by issuer)
#add_proof("Issuer 0")

In [8]:
tx = send_coins(vals=[ [0, 2, -2],[0, -1, 1] ], file_name="Bank 1", audit_pk = public_key ) # this is the broadcast  (tx + approve tx by issuer)
add_proof("Bank 1")

66f660af4a085b8c4a4a0fc0485eb98420fc79b2efbc0a3fe2f715ccbc4f31d7
add proof Bank 1


[[<pyledger.ledger.MakeLedger.Cell at 0x1114821d0>,

In [9]:
add_proof('Issuer 0')
add_proof('Bank 2')

add proof Issuer 0
add proof Bank 2


[[<pyledger.ledger.MakeLedger.Cell at 0x111b52dd0>,

In [10]:
vote_tx("Issuer 0")
vote_tx("Bank 1")
vote_tx("Bank 2")

In [11]:
tx = finalize_tx("Issuer 0")

In [12]:
check_balance("Issuer 0")

------- Issuer 0 -------
x 996
y 996
-----------------------


[996, 996]

In [13]:
check_balance("Bank 1")

------- Bank 1 -------
x 2
y 3
-----------------------


[2, 3]

In [14]:
check_balance("Bank 2")

------- Bank 2 -------
x 2
y 1
-----------------------


[2, 1]